In [1]:
import numpy as np

# Module

**Module** is an abstract class which defines fundamental methods necessary for a training a neural network. You do not need to change anything here, just read the comments.

In [2]:
class Module(object):
    
    """
    Basically, you can think of a module as of a something (black box) 
    which can process `input` data and produce `ouput` data.
    This is like applying a function which is called `forward`: 
        
        output = module.forward(input)
    
    The module should be able to perform a backward pass: to differentiate the `forward` function. 
    More, it should be able to differentiate it as part of chain (chain rule).
    The latter implies there is a gradient from previous step of a chain rule. 
    
        gradInput = module.backward(input, gradOutput)
    """
    
    def __init__ (self):
        
        self.output = None
        self.gradInput = None
        self.training = True
    
    def forward(self, input):
        
        """
        Takes an input object, and computes the corresponding output of the module.
        """
        
        return self.updateOutput(input)

    def backward(self,input, gradOutput):
        
        """
        Performs a backpropagation step through the module, with respect to the given input.
        
        This includes 
         - computing a gradient w.r.t. `input` (is needed for further backprop),
         - computing a gradient w.r.t. parameters (to update parameters while optimizing).
        """
        
        self.updateGradInput(input, gradOutput)
        self.accGradParameters(input, gradOutput)
        return self.gradInput
    

    def updateOutput(self, input):
        
        """
        Computes the output using the current parameter set of the class and input.
        This function returns the result which is stored in the `output` field.
        
        Make sure to both store the data in `output` field and return it. 
        """
        
        # The easiest case:
            
        # self.output = input 
        # return self.output
        
        pass

    def updateGradInput(self, input, gradOutput):
        
        """
        Computing the gradient of the module with respect to its own input. 
        This is returned in `gradInput`. Also, the `gradInput` state variable is updated accordingly.
        
        The shape of `gradInput` is always the same as the shape of `input`.
        
        Make sure to both store the gradients in `gradInput` field and return it.
        """
        
        # The easiest case:
        
        # self.gradInput = gradOutput 
        # return self.gradInput
        
        pass   
    
    def accGradParameters(self, input, gradOutput):
        
        """
        Computing the gradient of the module with respect to its own parameters.
        No need to override if module has no parameters (e.g. ReLU).
        """
        
        pass
    
    def zeroGradParameters(self): 
        
        """
        Zeroes `gradParams` variable if the module has params.
        """
        
        pass
        
    def getParameters(self):
        
        """
        Returns a list with its parameters. 
        If the module does not have parameters return empty list. 
        """
        
        return []
        
    def getGradParameters(self):
        
        """
        Returns a list with gradients with respect to its parameters. 
        If the module does not have parameters return empty list. 
        """
        
        return []
    
    def _set_training_mode(self):
        
        """
        Sets training mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        
        self.training = True
    
    def _set_evaluation_mode(self):
        
        """
        Sets evaluation mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        
        self.training = False
    
    def __repr__(self):
        
        """
        Pretty printing. Should be overrided in every module if you want 
        to have readable description. 
        """
        
        return "Module"

# Sequential container

This is a sequential module which is basically a container for storing layers and doing forwads/backward passes through them.

Your task is to define a **forward and backward** pass procedures.

In [3]:
class Sequential(Module):
    
    """
    This class implements a container, which processes `input` data sequentially. 

    `input` is processed by each module (layer) in self.modules consecutively.
    The resulting array is called `output`. 
    """
    
    def __init__ (self):
        
        super(Sequential, self).__init__()
        self.modules = []
   
    def add(self, module):
        
        """
        Adds a module to the container.
        """
        
        self.modules.append(module)

    def updateOutput(self, input):
        
        """
        Basic workflow of FORWARD PASS:

            y_0    = module[0].forward(input)
            y_1    = module[1].forward(y_0)
            ...
            output = module[n-1].forward(y_{n-2})   


        Just write a loop. 
        """

        #==================================================#
        
        current_input = input
        
        for layer in self.modules:
            current_input = layer.forward(current_input)
        
        self.output = current_input
        
        #==================================================#
        
        return self.output

    def backward(self, input, gradOutput):
        
        """
        Workflow of BACKWARD PASS:
            
            g_{n-1} = module[n-1].backward(y_{n-2}, gradOutput)
            g_{n-2} = module[n-2].backward(y_{n-3}, g_{n-1})
            ...
            g_1 = module[1].backward(y_0, g_2)   
            gradInput = module[0].backward(input, g_1)   
             
             
        !!!
                
        To each module you need to provide the input, module saw while forward pass, 
        it is used while computing gradients.
        Make sure that the input for `i-th` layer the output of `module[i]` 
        (just the same input as in forward pass) and NOT `input` to this Sequential module. 
        
        !!!
        
        """
        
        #==================================================#
        
        current_gradOutput = gradOutput
        
        for layer_num in range(len(self.modules) - 1, 0, -1):
            current_gradOutput = self.modules[layer_num].backward(self.modules[layer_num - 1].output, 
                                                                  current_gradOutput)
        
        self.gradInput = self.modules[0].backward(input, 
                                                  current_gradOutput)
        
        #==================================================#
        
        return self.gradInput
      

    def zeroGradParameters(self): 
        
        for module in self.modules:
            module.zeroGradParameters()
    
    def getParameters(self):
        
        """
        Should gather all parameters in a list.
        """
        
        return [x.getParameters() for x in self.modules]
    
    def getGradParameters(self):
        
        """
        Should gather all gradients w.r.t parameters in a list.
        """
        
        return [x.getGradParameters() for x in self.modules]
    
    def _set_training_mode(self):
        
        """
        Sets training mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        
        super(Sequential, self)._set_training_mode()
        for layer in self.modules:
            layer._set_training_mode()
    
    def _set_evaluation_mode(self):
        
        """
        Sets evaluation mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        
        super(Sequential, self)._set_evaluation_mode()
        for layer in self.modules:
            layer._set_evaluation_mode()
    
    def __repr__(self):
        
        string = "".join([str(x) + '\n' for x in self.modules])
        return string
    
    def __getitem__(self,x):
        
        return self.modules.__getitem__(x)

# Layers

## Linear

Below there is the simplest possible layer: fully-connected. You are to implement both forward and backward passes. 

Layer should work with 2D tensors of the following shapes:
- input: **`batch_size X n_feats1`**
- output: **`batch_size X n_feats2`**

In [4]:
class Linear(Module):
    
    """
    A module which applies a linear transformation 
    A common name is fully-connected layer, InnerProductLayer in caffe. 
    
    The module should work with 2D input of shape (n_samples, n_feature).
    """
    
    def __init__(self, n_in, n_out):
        
        super(Linear, self).__init__()
       
        # This is a Xavier initialization
        stdv = 1. / np.sqrt(n_in)
        self.W = np.random.uniform(-stdv, stdv, size = (n_out, n_in))
        self.b = np.random.uniform(-stdv, stdv, size = n_out)
        
        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)
        
    def updateOutput(self, input):
        
        #==================================================#
        
        self.output = np.dot(input, self.W.T) + self.b
        
        #==================================================#
        
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        
        #==================================================#
        
        self.gradInput = np.dot(gradOutput, self.W)
        
        #==================================================#
        
        return self.gradInput
    
    def accGradParameters(self, input, gradOutput):
        
        #==================================================#
        
        self.gradW = np.dot(gradOutput.T, input)
        self.gradb = np.sum(gradOutput, axis=0, keepdims=False)
        
        #==================================================#
        
        return [self.gradW, self.gradb]
    
    def zeroGradParameters(self):
        
        self.gradW.fill(0)
        self.gradb.fill(0)
        
    def getParameters(self):
        
        return [self.W, self.b]
    
    def getGradParameters(self):
        
        return [self.gradW, self.gradb]
    
    def __repr__(self):
        
        s = self.W.shape
        q = 'Linear %d -> %d' % (s[1], s[0])
        return q

## Softmax

Acitvation functions are also layers in this approach. Below one can find an implementation of `Softmax` layer which works with the following shapes: 
- input:   **`batch_size X n_feats`**
- output: **`batch_size X n_feats`**

In [5]:
class SoftMax(Module):
    
    def __init__(self):
        
         super(SoftMax, self).__init__()
    
    def updateOutput(self, input):
        
        # start with normalization for numerical stability
        self.output = np.subtract(input, input.max(axis=1, keepdims=True))
        
        self.output = np.exp(self.output)
        self.output = np.divide(self.output, self.output.sum(axis=1, keepdims=True))
        return self.output
    
    def updateGradInput(self, input, gradOutput):

        self.gradInput = self.output * (gradOutput - np.sum(self.output * gradOutput, axis=1, keepdims=True))
        
        return self.gradInput
    
    def __repr__(self):
        
        return "SoftMax"

## Batch mean subtraction

One of the most significant recent ideas that impacted NNs a lot is [**Batch normalization**](http://arxiv.org/abs/1502.03167). The idea is simple, yet effective: the features should be whitened $mean = 0, std = 1$ all the way through NN. This improves the convergence for deep models and allows the training procedure ends in days but not weeks. 

Below one can find an implementation of BatchNorm simplification which is **mean subtraction**. That is, the module calculates mean value over the batch for every feature (every column) and subtract it from the whole batch.

Note, that one need to estimate the mean over the dataset to be able to predict on test examples. The right way is to create a variable which will hold smoothed mean over batches (exponential smoothing works good) and use it when forwarding test examples.

When training calculate rolled mean for test stage as folows: 
```
    mean_to_subtract = self.old_mean * alpha + batch_mean * (1 - alpha)
```
when evaluating (`self.training == False`) set $alpha = 1$.

Input and output shapes are the following:
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

In [6]:
class BatchMeanSubtraction(Module):
    
    def __init__(self, alpha = 0.9):

        super(BatchMeanSubtraction, self).__init__()
        
        self.alpha = alpha
        self.old_mean = None 
        
    def updateOutput(self, input):
        
        if self.training:
            batch_mean = np.mean(input, axis=0, keepdims=True)

            self.output = input - batch_mean

            if self.old_mean is not None:
                self.old_mean = self.old_mean * self.alpha + batch_mean * (1. - self.alpha)
            else:
                self.old_mean = batch_mean
        else:
            self.output = input - self.old_mean
        
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        
# https://kratzert.github.io/2016/02/12/understanding-the-gradient-flow-through-the-batch-normalization-layer.html
        
        if self.training:
            self.gradInput = gradOutput - np.mean(gradOutput, axis=0, keepdims=True)
        else:
            self.gradInput = gradOutput
        
        return self.gradInput
    
    def __repr__(self):
        
        return "BatchMeanNormalization"

## Dropout

Implement [**dropout**](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf). The idea and implementation is really simple: just multimply the input by $Bernoulli(p)$ mask. 

This is a very cool regularizer. In fact, when you see your net is overfitting try to add more dropout.

**Note** that while training (`self.training == True`) it should sample a mask on each iteration (for every batch). When testing this module should implement identity transform i.e. `self.output = input`.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

In [7]:
class Dropout(Module):
    
    def __init__(self, p=0.5):
        
        super(Dropout, self).__init__()
        
        self.p = p
        self.mask = None
        
    def updateOutput(self, input):
        
        #==================================================#
        
        # In classical formulation of dropout 
        # p should be a probability of dropping the node
        # so one should consider Bernoulli(1-p) mask
        
        # default behaviour
        self.output = input
        # if training stage then apply dropout
        if self.training:
            # generate mask
            self.mask = np.random.rand(*input.shape) < 1 - self.p
            # scaling
            self.mask = self.mask / (1. - self.p)
            # applying mask
            self.output *= self.mask
            
        #==================================================#
        
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        
        #==================================================#
        
        # default behaviour
        self.gradInput = gradOutput
        # if training stage then apply dropout
        if self.training:
            self.gradInput *= self.mask
            
        #==================================================#
        
        return self.gradInput
        
    def __repr__(self):
        
        return "Dropout %0.2f" % (self.p)

## ReLU

Here's the complete example for the **Rectified Linear Unit** non-linearity (aka **ReLU**): 

In [8]:
class ReLU(Module):
    
    def __init__(self):
        
         super(ReLU, self).__init__()
    
    def updateOutput(self, input):
        
        self.output = np.maximum(input, 0)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        
        self.gradInput = np.multiply(gradOutput , input > 0)
        return self.gradInput
    
    def __repr__(self):
        
        return "ReLU"

## Leaky ReLU

Implement [**Leaky Rectified Linear Unit**](http://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29%23Leaky_ReLUs). Expriment with slope. 

In [9]:
class LeakyReLU(Module):
    
    def __init__(self, slope = 0.03):
        
        super(LeakyReLU, self).__init__()
            
        self.slope = slope
        
    def updateOutput(self, input):
        
        #==================================================#
        
        self.output = np.maximum(input, self.slope * input)
        
        #==================================================#
        
        return  self.output
    
    def updateGradInput(self, input, gradOutput):
        
        #==================================================#
        
        mask = input > 0
        self.gradInput = gradOutput * (~mask * self.slope + mask * 1.)
        
        #==================================================#
        
        return self.gradInput
    
    def __repr__(self):
        
        return "LeakyReLU %0.2f" % (self.slope)

## ELU

Below [**Exponential Linear Units**](http://arxiv.org/abs/1511.07289) activation is implemented for you

In [10]:
class ELU(Module):
    
    def __init__(self, alpha = 1.0):
        
        super(ELU, self).__init__()
        
        self.alpha = alpha
        
    def updateOutput(self, input):
        
        mask = input > 0
        self.output = mask * input + ~mask * self.alpha * (np.exp(input) - 1.)
        return  self.output
    
    def updateGradInput(self, input, gradOutput):
        
        mask = input > 0
        self.gradInput = mask * gradOutput + ~mask * (self.output + self.alpha) * gradOutput
        return self.gradInput
    
    def __repr__(self):
        
        return "ELU %0.2f" % (self.alpha)

## Softplus

Implement [**SoftPlus**](https://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29) activations. Look, how they look a lot like ReLU.

In [11]:
class SoftPlus(Module):
    
    def __init__(self):
        
        super(SoftPlus, self).__init__()
    
    def updateOutput(self, input):
        
        #==================================================#
        
        self.output = np.log(1. + np.exp(input))
        
        #==================================================#
        
        return  self.output
    
    def updateGradInput(self, input, gradOutput):
        
        #==================================================#
        
        self.gradInput = gradOutput * 1. / (1. + np.exp(-input))
        
        #==================================================#
        
        return self.gradInput
    
    def __repr__(self):
        
        return "SoftPlus"

# Criterions

Criterions are used to score the models answers. 

In [12]:
class Criterion(object):
    
    def __init__ (self):
        
        self.output = None
        self.gradInput = None
        
    def forward(self, input, target):
        
        """
            Given an input and a target, compute the loss function 
            associated to the criterion and return the result.
            
            For consistency this function should not be overrided,
            all the code goes in `updateOutput`.
        """
        
        return self.updateOutput(input, target)

    def backward(self, input, target):
        
        """
            Given an input and a target, compute the gradients of the loss function
            associated to the criterion and return the result. 

            For consistency this function should not be overrided,
            all the code goes in `updateGradInput`.
        """
        
        return self.updateGradInput(input, target)
    
    def updateOutput(self, input, target):
        
        """
        Function to override.
        """
        
        return self.output

    def updateGradInput(self, input, target):
        
        """
        Function to override.
        """
        
        return self.gradInput   

    def __repr__(self):
        
        """
        Pretty printing. Should be overrided in every module if you want 
        to have readable description. 
        """
        
        return "Criterion"

## Cross-entropy

Below there is an implementation of the **ClassNLLCriterion**. It implements [multiclass log loss](http://scikit-learn.org/stable/modules/model_evaluation.html#log-loss). Nevertheless there is a sum over `y` (target) in that formula, remember that targets are one-hot encoded. This fact simplifies the computations a lot. Note, that criterions are the only places, where you divide by batch size. 

In [13]:
class ClassNLLCriterion(Criterion):
    
    def __init__(self):
        
        super(ClassNLLCriterion, self).__init__()
        
    def updateOutput(self, input, target): 
        
        # Use this trick to avoid numerical errors
        
        eps = 1e-15 
        input_clamp = np.clip(input, eps, 1 - eps)
        
        N = target.shape[0]
        class_idxs = np.argmax(target, axis=1)
        self.output = - np.log(input_clamp[range(N), class_idxs])
        self.output = np.mean(self.output)
        
        return self.output

    def updateGradInput(self, input, target):
        
        # Use this trick to avoid numerical errors
        input_clamp = np.maximum(1e-15, np.minimum(input, 1 - 1e-15) )
        
        N = target.shape[0]
        K = target.shape[1]
        class_idxs = np.argmax(target, axis=1)
        
        self.gradInput = np.zeros((N, K))
        self.gradInput[range(N), class_idxs] = - 1. / N * 1. / input_clamp[range(N), class_idxs]
        
        return self.gradInput
    
    def __repr__(self):
        
        return "ClassNLLCriterion"

## MSE

You should implement **MSECriterion**, which is basic L2 norm usually used for regression

In [14]:
class MSECriterion(Criterion):
    
    def __init__(self):
        
        super(MSECriterion, self).__init__()
        
    def updateOutput(self, input, target):   
        
        #==================================================#
        
        self.output = np.mean(np.power(input - target, 2))
        
        #==================================================#
        
        return self.output 
 
    def updateGradInput(self, input, target):
        
        #==================================================#
        
        self.gradInput  = np.mean((input - target) * 2)
        
        #==================================================#
        
        return self.gradInput

    def __repr__(self):
        
        return "MSECriterion"